In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')

In [2]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [3]:
from sklearn.model_selection import KFold

In [4]:
import dask.dataframe as dd

In [5]:
gc.enable()

In [6]:
np.random.seed(51)

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
"""
%%time
train_metadata_kaggle = pd.read_csv('train_metadata_kaggle.csv')
test_metadata_kaggle = pd.read_csv('test_metadata_kaggle.csv')
"""

"\n%%time\ntrain_metadata_kaggle = pd.read_csv('train_metadata_kaggle.csv')\ntest_metadata_kaggle = pd.read_csv('test_metadata_kaggle.csv')\n"

In [9]:
%%time
train_metadata_kaggle = dd.read_csv('train_metadata_kaggle.csv')
test_metadata_kaggle = dd.read_csv('test_metadata_kaggle.csv')
train_metadata_kaggle = train_metadata_kaggle.compute()
test_metadata_kaggle = test_metadata_kaggle.compute()
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 51) (3492890, 50)
CPU times: user 1min 21s, sys: 5.5 s, total: 1min 26s
Wall time: 15.9 s


In [10]:
test_id = test_metadata_kaggle['object_id']

In [11]:
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 51) (3492890, 50)


In [12]:
test_metadata_kaggle.head()

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_diff,flux_dif2,flux_w_mean,flux_dif3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,mwebv
0,13,-12.680235,42.765503,3.997127,0.616561,9.149645,2.037355,0.691634,11.257108,2.461810,1.972973,1.718101,1.826388,0.157576,7806.412424,4.771625,1.896346e+05,5.396523,55.445738,13.871398,24.292155,2.282455,29.002872,37.684425,-0.247160,0.147622,92.039690,79.990817,10.567412,3.117684,164.640622,139.733762,4.512783,2.171455,300.547278,246.788411,2.951479,1.806797,408.305525,349.008202,0.094963,1.153076,324.515880,251.116361,0.072799,0.804929,119.8531,0.3193,0.0542,0.019
1,14,-11.142164,14.839427,0.884047,0.072856,3.399946,0.970525,0.690589,11.249375,2.457580,1.973559,1.717591,1.826703,0.012121,806.406927,11.486148,5.525817e+03,12.348124,25.981591,29.389389,6.852393,3.791608,22.708482,26.159787,0.099267,0.502325,13.685195,27.630359,13.429229,3.109318,31.012899,33.427074,13.769006,3.247873,56.042403,59.784625,1.750840,1.082798,83.561278,77.494564,-0.322108,-0.099957,84.725142,27.013154,-0.348744,0.132025,28.0053,0.6323,0.0179,0.018
2,17,-14.202744,16.761280,0.791032,0.458390,3.886578,0.377131,0.663680,11.278636,2.702947,2.184483,1.922641,1.802497,0.014205,784.835502,9.509911,4.124400e+03,9.923556,30.964024,39.143819,5.255113,5.892170,6.030862,46.450439,0.590786,-0.427429,53.366119,41.192678,4.127763,1.710048,58.607170,42.353288,8.642889,2.570936,26.975615,59.958041,3.484929,1.141959,57.222812,72.448018,2.085327,0.429582,76.240782,73.533783,0.063837,-0.175525,2.7628,0.8297,0.0605,0.016
3,23,-12.631923,28.061138,0.970396,0.465986,4.100713,2.177402,0.697639,11.305429,2.480364,2.003704,1.723367,1.821626,0.018182,876.027511,8.010597,8.293673e+03,9.227223,40.693061,41.934474,9.467365,4.298245,0.776942,14.926490,-0.580529,-0.058688,2.371129,22.476077,1.429683,-0.000571,34.447730,14.240274,1.223998,0.468594,58.911457,62.709899,4.269139,1.414380,115.918277,70.371095,7.377841,2.409871,112.547521,123.413344,1.793238,0.953107,3.0125,0.6533,0.1479,0.023
4,34,-13.239577,124.475609,4.580070,0.301366,19.862714,4.537527,0.679312,11.365292,2.746784,2.210577,1.930790,1.798629,0.065341,47612.580669,9.140568,4.815012e+06,10.251332,137.715186,30.068359,101.128982,1.361778,65.064185,55.191226,7.457425,2.135457,202.641918,191.410916,16.393710,4.133929,425.940451,414.455381,15.206929,4.036551,368.228927,360.630612,15.386561,4.030676,377.744797,301.983950,15.302549,3.992740,172.564336,289.757599,12.507289,3.513977,25.0572,0.4617,0.0122,0.023


In [13]:
%%time
train_metadata = dd.read_csv('train_metadata_final.csv')
test_metadata = dd.read_csv('test_metadata_final.csv')
train_metadata = train_metadata.compute()
test_metadata = test_metadata.compute()
print(train_metadata.shape,test_metadata.shape)

(7848, 233) (3492890, 232)
CPU times: user 4min 7s, sys: 14.6 s, total: 4min 22s
Wall time: 48.3 s


In [14]:
"""%%time
train_metadata = pd.read_csv('train_metadata_final.csv')
test_metadata = pd.read_csv('test_metadata_final.csv')"""

"%%time\ntrain_metadata = pd.read_csv('train_metadata_final.csv')\ntest_metadata = pd.read_csv('test_metadata_final.csv')"

In [15]:
y = train_metadata['target']
del train_metadata['target']
gc.collect()

518

In [16]:
will_be_deleted_columns = ['NG_min_flux','NG_max_flux','NG_std_flux','NG_mean_flux','NG_median_flux',
                          'NG_min_flux_err','NG_max_flux_err','NG_std_flux_err','NG_mean_flux_err','NG_median_flux_err',
                          'NG_mean_detected','hostgal_photoz','hostgal_photoz_err','mwebv']

In [17]:
%%time
train_metadata.drop(will_be_deleted_columns,inplace=True , axis=1)
test_metadata.drop(will_be_deleted_columns,inplace=True , axis=1)
gc.collect()

CPU times: user 3.88 s, sys: 5.89 s, total: 9.77 s
Wall time: 9.75 s


In [18]:
print(train_metadata.shape,test_metadata.shape)

(7848, 218) (3492890, 218)


In [19]:
test_metadata.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,distmod,NG_sum_flux_err,NG_sum_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_median_detected,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s,percent_p0_region_minus_4,percent_p0_region_minus_3,percent_p0_region_minus_2,percent_p0_region_minus_1,percent_p0_region_plus_1,percent_p0_region_plus_2,percent_p0_region_plus_3,percent_p0_region_plus_4,percent_p1_region_minus_4,percent_p1_region_minus_3,percent_p1_region_minus_2,percent_p1_region_minus_1,percent_p1_region_plus_1,percent_p1_region_plus_2,percent_p1_region_plus_3,percent_p1_region_plus_4,percent_p2_region_minus_4,percent_p2_region_minus_3,percent_p2_region_minus_2,percent_p2_region_minus_1,percent_p2_region_plus_1,percent_p2_region_plus_2,percent_p2_region_plus_3,percent_p2_region_plus_4,percent_p3_region_minus_4,percent_p3_region_minus_3,percent_p3_region_minus_2,percent_p3_region_minus_1,percent_p3_region_plus_1,percent_p3_region_plus_2,percent_p3_region_plus_3,percent_p3_region_plus_4,percent_p4_region_minus_4,percent_p4_region_minus_3,percent_p4_region_minus_2,percent_p4_region_minus_1,percent_p4_region_plus_1,percent_p4_region_plus_2,percent_p4_region_plus_3,percent_p4_region_plus_4,percent_p5_region_minus_4,percent_p5_region_minus_3,percent_p5_region_minus_2,percent_p5_region_minus_1,percent_p5_region_plus_1,percent_p5_region_plus_2,percent_p5_region_plus_3,percent_p5_region_plus_4
0,13,34.44,-5.230,170.0,-59.97,1,0.3047,41.12,812.39730,1319.05190,330.0,0.3650,52.0,0.0,1.122888,3.087183,0.424485,146.99579,2.0416

In [20]:
#for i,j in enumerate(list(train_metadata.columns.values)):
#    print(i,j)

# Predicting hostgal_specz from features

In [21]:
NG_columns = [x for x in list(train_metadata.columns.values) if x[:2] =="NG"]

In [22]:
A0_columns = [x for x in list(train_metadata.columns.values) if x[:2] =="A0"]

In [23]:
print(NG_columns)

['NG_sum_flux_err', 'NG_sum_flux', 'NG_count_detected', 'NG_std_detected', 'NG_sum_detected', 'NG_median_detected']


In [24]:
#ignored_columns = ['ra','decl','gal_l','gal_b','ddf','hostgal_specz','distmod']

In [25]:
temp = train_metadata_kaggle.merge(right = train_metadata ,how='left',on = 'object_id')

In [26]:
temp = temp[sorted(temp.columns)]

In [27]:
del temp['target']

In [28]:
%%time
temp_test = test_metadata_kaggle.merge(right = test_metadata ,how='left',on = 'object_id')

CPU times: user 8.88 s, sys: 10.1 s, total: 18.9 s
Wall time: 18.9 s


In [29]:
temp_test = temp_test[sorted(temp_test.columns)]

In [30]:
print(temp.shape,temp_test.shape)

(7848, 267) (3492890, 267)


In [31]:
sorted(temp.columns) == sorted(temp_test.columns)

True

In [32]:
temp.head()

,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,A0_count_detected,A0_max_flux,A0_max_flux_err,A0_mean_detected,A0_mean_flux,A0_mean_flux_err,A0_median_detected,A0_median_flux,A0_median_flux_err,A0_min_flux,A0_min_flux_err,A0_std_detected,A0_std_flux,A0_std_flux_err,A0_sum_detected,A0_sum_flux,A0_sum_flux_err,A1_count_detected,A1_max_flux,A1_max_flux_err,A1_mean_detected,A1_mean_flux,A1_mean_flux_err,A1_median_detected,A1_median_flux,A1_median_flux_err,A1_min_flux,A1_min_flux_err,A1_std_detected,A1_std_flux,A1_std_flux_err,A1_sum_detected,A1_sum_flux,A1_sum_flux_err,A2_count_detected,A2_max_flux,A2_max_flux_err,A2_mean_detected,A2_mean_flux,A2_mean_flux_err,A2_median_detected,A2_median_flux,A2_median_flux_err,A2_min_flux,A2_min_flux_err,A2_std_detected,A2_std_flux,A2_std_flux_err,A2_sum_detected,A2_sum_flux,A2_sum_flux_err,A3_count_detected,A3_max_flux,A3_max_flux_err,A3_mean_detected,A3_mean_flux,A3_mean_flux_err,A3_median_detected,A3_median_flux,A3_median_flux_err,A3_min_flux,A3_min_flux_err,A3_std_detected,A3_std_flux,A3_std_flux_err,A3_sum_detected,A3_sum_flux,A3_sum_flux_err,A4_count_detected,A4_max_flux,A4_max_flux_err,A4_mean_detected,A4_mean_flux,A4_mean_flux_err,A4_median_detected,A4_median_flux,A4_median_flux_err,A4_min_flux,A4_min_flux_err,A4_std_detected,A4_std_flux,A4_std_flux_err,A4_sum_detected,A4_sum_flux,A4_sum_flux_err,A5_count_detected,A5_max_flux,A5_max_flux_err,A5_mean_detected,A5_mean_flux,A5_mean_flux_err,A5_median_detected,A5_median_flux,A5_median_flux_err,A5_min_flux,A5_min_flux_err,A5_std_detected,A5_std_flux,A5_std_flux_err,A5_sum_detected,A5_sum_flux,A5_sum_flux_err,NG_count_detected,NG_median_detected,NG_std_detected,NG_sum_detected,NG_sum_flux,NG_sum_flux_err,ddf,decl,detected_mean,distmod,flux_by_flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_dif2,flux_dif3,flux_diff,flux_err_max,flux_err_mean,flux_err_median,flux_err_min,flux_err_skew,flux_err_std,flux_max,flux_mean,flux_median,flux_min,flux_ratio_sq_skew,flux_ratio_sq_sum,flux_skew,flux_std,flux_w_mean,gal_b,gal_l,hostgal_photoz,hostgal_photoz_err,hostgal_specz,mjd_diff_det,mwebv,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s,object_id,p0_region_minus_1,p0_region_minus_2,p0_region_minus_3,p0_region_minus_4,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_1,p1_region_minus_2,p1_region_minus_3,p1_region_minus_4,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_1,p2_region_minus_2,p2_region_minus_3,p2_region_minus_4,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_1,p3_region_minus_2,p3_region_minus_3,p3_region_minus_4,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_1,p4_region_minus_2,p4_region_minus_3,p4_region_minus_4,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_1,p5_region_minus_2,p5_region_minus_3,p5_region_minus_4,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,percent_p0_region_minus_1,percent_p0_region_minus_2,percent_p0_region_minus_3,percent_p0_region_minus_4,percent_p0_region_plus_1,percent_p0_region_plus_2,percent_p0_region_plus_3,percent_p0_region_plus_4,percent_p1_region_minus_1,percent_p1_region_minus_2,percent_p1_region_minus_3,percent_p1_region_minus_4,percent_p1_region_plus_1,percent_p1_re

In [33]:
filled_hostgal_specz = temp_test[temp_test['hostgal_specz'].notnull()]

In [34]:
unfilled_hostgal_specz = temp_test[temp_test['hostgal_specz'].isnull()]

In [35]:
unfilled_hostgal_specz_id =  unfilled_hostgal_specz['object_id']
del unfilled_hostgal_specz['hostgal_specz'] , unfilled_hostgal_specz['object_id']

In [36]:
print(unfilled_hostgal_specz_id.shape)

(3372497,)


In [37]:
%%time
filled_hostgal_specz = pd.concat([temp,filled_hostgal_specz],axis=0)

CPU times: user 129 ms, sys: 108 ms, total: 237 ms
Wall time: 238 ms


In [38]:
print(filled_hostgal_specz.shape)

(128241, 267)


In [39]:
%%time
y_hostgal_specz = filled_hostgal_specz['hostgal_specz']
filled_hostgal_specz_id = filled_hostgal_specz['object_id']
del filled_hostgal_specz['hostgal_specz'],filled_hostgal_specz['object_id']
gc.collect()

CPU times: user 54.7 ms, sys: 12.9 ms, total: 67.7 ms
Wall time: 74.3 ms


In [40]:
filled_hostgal_specz = filled_hostgal_specz.reset_index(drop=True)
y_hostgal_specz = y_hostgal_specz.reset_index(drop=True)

In [41]:
print(filled_hostgal_specz.shape,y_hostgal_specz.shape,unfilled_hostgal_specz.shape)

(128241, 265) (128241,) (3372497, 265)


In [ ]:
"""%%time
for i in [0.005,0.01,0.03,0.05,0.07]:
    oof_preds = np.zeros(filled_hostgal_specz.shape[0])
    folds = KFold(n_splits=5, shuffle=True, random_state=51)
    clfs = []
    importances = pd.DataFrame()
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'root_mean_squared_error',
        'learning_rate': 0.03,
        'subsample': 0.5,
        'colsample_bytree': 0.5,
        'reg_alpha': 0.01,
        'reg_lambda': .01,
        'min_split_gain': 0.01,
        'min_child_weight': 10,
        'n_estimators': 1000,
        'silent': True,
        'verbose': -1,
        'max_depth': 5
    }
    for fold_, (trn_, val_) in enumerate(folds.split(filled_hostgal_specz)):
        trn_x, trn_y = filled_hostgal_specz.iloc[trn_], y_hostgal_specz.iloc[trn_]
        val_x, val_y = filled_hostgal_specz.iloc[val_], y_hostgal_specz.iloc[val_]

        clf = lgb.LGBMRegressor(**lgb_params)
        clf.fit(
            trn_x, trn_y,
            eval_set=[(trn_x, trn_y), (val_x, val_y)],
            verbose=False,
            early_stopping_rounds=50)

        oof_preds[val_] = clf.predict(val_x, num_iteration=clf.best_iteration_)

        imp_df = pd.DataFrame()
        imp_df['feature'] = filled_hostgal_specz.columns
        imp_df['gain'] = clf.feature_importances_
        imp_df['fold'] = fold_ + 1
        importances = pd.concat([importances, imp_df], axis=0, sort=False)

        clfs.append(clf)
    print(i,sqrt(mean_squared_error(y_hostgal_specz, pd.Series(oof_preds))) )"""

In [42]:
%%time

oof_preds = np.zeros(filled_hostgal_specz.shape[0])
folds = KFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'root_mean_squared_error',
    'learning_rate': 0.03,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'reg_alpha': 0.01,
    'reg_lambda': .01,
    'min_split_gain': 0.01,
    'min_child_weight': 10,
    'n_estimators': 1000,
    'silent': True,
    'verbose': -1,
    'max_depth': 5
}
for fold_, (trn_, val_) in enumerate(folds.split(filled_hostgal_specz)):
    trn_x, trn_y = filled_hostgal_specz.iloc[trn_], y_hostgal_specz.iloc[trn_]
    val_x, val_y = filled_hostgal_specz.iloc[val_], y_hostgal_specz.iloc[val_]

    clf = lgb.LGBMRegressor(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        verbose=False,
        early_stopping_rounds=50)

    oof_preds[val_] = clf.predict(val_x, num_iteration=clf.best_iteration_)

    """imp_df = pd.DataFrame()
    imp_df['feature'] = filled_hostgal_specz.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)"""

    clfs.append(clf)
print(sqrt(mean_squared_error(y_hostgal_specz, pd.Series(oof_preds))) )

0.06796634812739077
CPU times: user 9min 19s, sys: 4.29 s, total: 9min 24s
Wall time: 1min 22s


In [44]:
%%time
test_pred0 = pd.DataFrame()
test_pred1 = pd.DataFrame()
test_pred2 = pd.DataFrame()
test_pred3 = pd.DataFrame()
test_pred4 = pd.DataFrame()

list_of_df = [test_pred0,test_pred1,test_pred2,test_pred3,test_pred4]

for num,c in enumerate(clfs):
    print(num)
    for k in range(0,len(unfilled_hostgal_specz),100000):
        test_pred = pd.DataFrame( c.predict(unfilled_hostgal_specz[ k:k+100000] ) )
        list_of_df[num] = pd.concat([list_of_df[num],test_pred],axis=0)
        del test_pred

0
1
2
3
4
CPU times: user 24min 16s, sys: 1min 4s, total: 25min 20s
Wall time: 5min 6s


In [57]:
list_of_df[0].head()

,0
0,0.633108
1,0.810001
2,0.580423
3,0.804671
4,0.653700


In [59]:
test_pred = pd.DataFrame()
test_pred = (list_of_df[0] + list_of_df[1] + list_of_df[2] + list_of_df[3] + list_of_df[4])/5

In [60]:
test_pred.head()

,0
0,0.636228
1,0.818608
2,0.580034
3,0.807865
4,0.655916


In [61]:
test_pred.columns = ['hostgal_specz']

In [62]:
test_pred.index= unfilled_hostgal_specz_id

In [63]:
y_hostgal_specz.index = filled_hostgal_specz_id

In [64]:
test_pred = test_pred['hostgal_specz']

In [65]:
test_pred.head()

object_id
14    0.636228
17    0.818608
23    0.580034
35    0.807865
43    0.655916
Name: hostgal_specz, dtype: float64

In [49]:
y_hostgal_specz.head()

0    0.0000
1    1.8180
2    0.2320
3    0.3037
4    0.1934
Name: hostgal_specz, dtype: float64

In [66]:
written_file = pd.concat([y_hostgal_specz,test_pred],axis=0)

In [70]:
written_file.shape[0] == train_metadata.shape[0] + test_metadata.shape[0]

True

In [68]:
written_file.to_csv('hostgal_specz.csv',index=True)